## Setup imports

In [ ]:
import os
import shutil
import yaml
import pandas as pd

## Load useful data

As described in `readme.md`, we manually labeled 1126 frames in order to build the detection model.
Please download the manually labeled bounding boxes from [google drive](https://drive.google.com/file/d/1iO4bXTGdhRLIoxIKS6P_nNAgI_1Fp_Vg/view?usp=sharing), the uncompressed folder `labels` is saved into `label_14_tools_yolo_640_blur/`.

In [ ]:
# please run `preprocess_detect_scene_and_split_fold.ipynb` first to produce this file
df = pd.read_csv('train_fold_balanced.csv')
dataset_dir = "/raid/label_14_tools_yolo_640_blur/"
labels_dir = os.path.join(dataset_dir, "labels")
# please run `preprocess_prepare_detection_dataset.ipynb` first to produce frames
images_dir = "/raid/surg/image640_blur/"
yolo_image_dir = os.path.join(dataset_dir, "images")

os.makedirs(yolo_image_dir, exist_ok=True)

## Prepare images

Copy video extracted frames into yolo dataset dir

In [ ]:
for label in os.listdir(labels_dir):
    image = label.replace("txt", "jpg")

    image_src = os.path.join(images_dir, image)
    image_dst = os.path.join(yolo_image_dir, image)
    shutil.copy(image_src, image_dst)

## Prepare yaml files

In [ ]:
df_samples = pd.DataFrame.from_dict(os.listdir(yolo_image_dir))
df_samples.columns = ["image_name"]
df_samples["clip_name"] = df_samples["image_name"].apply(lambda x: "clip_" + x.split("_")[1])
df = df.merge(df_samples, on="clip_name", how="right")

In [ ]:
label_dict = {
    "bipolar dissector": 0,
    "bipolar forceps": 1,
    "cadiere forceps": 2,
    "clip applier": 3,
    "force bipolar": 4,
    "grasping retractor": 5,
    "monopolar curved scissors": 6,
    "needle driver": 7,
    "permanent cautery hook/spatula": 8,
    "prograsp forceps": 9,
    "stapler": 10,
    "suction irrigator": 11,
    "tip-up fenestrated grasper": 12,
    "vessel sealer": 13,
}

In [ ]:
for i in range(5):
    df_train = df[df["fold"] != i]
    df_val = df[df["fold"] == i]

    with open(os.path.join(dataset_dir, 'train_fold{}.txt'.format(i)), 'w') as f:
        for path in df_train.image_name.tolist():
            full_path = os.path.join(dataset_dir, "images", path)
            f.write(full_path + '\n')

    with open(os.path.join(dataset_dir, 'val_fold{}.txt'.format(i)), 'w') as f:
        for path in df_val.image_name.tolist():
            full_path = os.path.join(dataset_dir, "images", path)
            f.write(full_path + '\n')
        print(len(df_train.image_name.tolist()), len(df_val.image_name.tolist()))

    data = {
        "path": dataset_dir,
        "train": os.path.join(dataset_dir, 'train_fold{}.txt'.format(i)),
        "val": os.path.join(dataset_dir, 'val_fold{}.txt'.format(i)),
        "nc": len(list(label_dict.keys())),
        "names": list(label_dict.keys()),
    }

    with open(os.path.join(dataset_dir, 'surg_14cls_fold{}.yaml'.format(i)), 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)